# IMPORTS

In [515]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import binom
###--------- PANDAS - EXIBIR TODAS COLUNAS ----###

pd.set_option('display.max_columns', None)


###--------- ESTILIZAÇÃO DO NOTEBOOK ---------###

from IPython.core.display import display, HTML

# retira a margem do notebook
display(HTML("<style>.container { width:100% !important; }</style>"))

# font do texto markdown
display(HTML("<style>h1 { font-size:23px !important; }</style>"))
display(HTML("<style>h2 { font-size:20px !important; }</style>"))
display(HTML("<style>h3 { font-size:17px !important; }</style>"))
display(HTML("<style>h4 { font-size:16px !important; }</style>"))
display(HTML("<style>p { font-size:16px !important; }</style>"))

# tamanho da fonte da tabela
display(HTML("<style>th { font-size:15px !important; }</style>"))
display(HTML("<style>td { font-size:15px !important; }</style>"))

# font do codigo 
display(HTML("<style>span { font-size:16px !important; }</style>"))

# Introdução

Se você realizou uma boa coleta e limpeza de dados, e quer melhorar ainda mais a acurácia do seu modelo,<br>
ou talvez reduzir o seu tempo de treinamento, a Seleção de Features é o caminho ideal para você. 

Por meio desse processo é possível selecionar as variáveis mais relevantes para o seu modelo, eliminando <br>
as features que não agregam para a predição do problema.


# Por que selecionar features ?

**1-)** <b>Ganho de velocidade no treinamento do modelo</b>. Ao eliminar features não relevantes <br>
ocorre uma redução de dimensionalidade do problema, reduzindo o tempo de treinamento.

**2-)** <b>Melhora da performance do modelo</b>. Ao remover as features que não contribuem para a <br>
predição do modelo, tornamos o nosso modelo mais simples e, consequentemente, reduzimos o ruído dos <br>
nossos dados e a chance de ocorrer overfitting.

# Seleção de Features vs Redução de Dimensionalidade

# Métodos de Seleção de Features

Os principais métodos de seleção de Features se agrupam em 3 tipos: <b>Filter Methods</b>, <b>Wrapper Methods</b> e <b>Embedded Methods</b>. 

## 1-) Filter Methods

## 2-) Wrapper Methods

## 3-) Embedded Methods

# Algoritmo Boruta

O algoritmo Boruta é um Wrapper Method, que seleciona automaticamente as features mais relevantes <br>
para o seu modelo. Uma grande dificuldade de outros métodos de Seleção de Features é a definição <br>
de um limite para determinar qual feature é ou não relevante, esse algoritmo resolve isso para você.

Para entender melhor o porquê desse algoritmo ser um ótimo selecionador de features relevantes, <br> 
vamos implementá-lo do zero. Para fins didáticos vamos criar um cenário hipotético, com dados fictícios.

# Cenário Hipotético

Uma loja de acessórios para pets fez uma pesquisa com algumas pessoas para determinar quais <br>
delas estariam interessadas nos produtos dessa loja.

A partir da pesquisa foram obtidos os seguintes dados: 

**1-)** A quantidade de pets que a pessoa tem.

**2-)** O salário.

**3-)** A idade.

**4-)** O peso.

**5-)** A altura do pai.

**6-)** Se a pessoa apresenta interesse em comprar um produto para seu pet.

## Intuição

Seguindo a nossa intuição, há uma suspeita de que a <b>quantidade de pets</b> que a pessoa tem <br>
é uma informação relevante para o modelo, e que a <b>altura do pai da pessoa</b> não tem valor <br> 
nenhum para a nossa predição.

Vale lembrar que o nosso dataset contém apenas 5 features, mas o que aconteceria se tivéssemos <br>
dezenas, senão centenas de variáveis para analisar ? Certamente perderíamos a capacidade de intuir <br>
sobre todos os dados e surgiriam muitas dúvidas como:


**1-)** Quais das features são relevantes para a modelagem do problema ?

**2-)** Qual critério utilizar para selecionar as features mais relevantes ?

Vamos responder essas dúvidas com o algoritmo Boruta.

## Sobre os dados do problema

O dataset criado apresenta 5 features, contendo as informações de 15 pessoas entrevistadas.

A idade, o peso e a altura seguem o comportamento de uma distribuição normal e foram criadas <br>
de modo a não influenciarem na variável resposta, já a quantidade de pets e a faixa salarial <br>
foram construídas de modo a influenciar na variável target.

Portanto, ao final do treinamento do Boruta é esperado que a quantidade de pets e a faixa <br>
salarial sejam as features consideradas relevantes, e as demais descartadas.

### Detalhes acerca da criação do dataset

In [375]:
# Vamos assumir que a idade, o peso e a altura da mãe estão normalmente distribuídos
quant_pets = np.random.normal(1.5, 1, 15)
faixa_salarial = np.random.normal(5000, 2000, 15)
faixa_salarial_mean = np.mean( faixa_salarial )
idade = np.random.normal(40, 12, 15)
peso =  np.random.normal(80, 8, 15)
altura_do_pai = np.random.normal(1.65, 0.07, 15)

# Convertendo a idade e o peso para números inteiros
quant_pets = quant_pets.astype(int)
faixa_salarial = faixa_salarial.astype(int)
idade = idade.astype(int)
peso = peso.astype(int)

# Arredondando para duas casas decimais
altura_do_pai = np.round( altura_do_pai, 2 )

x = pd.DataFrame({
                    'quant_pets': quant_pets,
                    'faixa_salarial': faixa_salarial,
                    'idade': idade,
                    'peso': peso,
                    'altura_do_pai': altura_do_pai,
    })

y = np.zeros((15,)) + 1
y = y * np.array( x['quant_pets'] )

y = pd.Series(y, name='interesse')
y = y.apply(lambda x: 0 if x==0 else 1)

drop_x = x['faixa_salarial'].apply(lambda x: 0 if x < faixa_salarial_mean else 1)
y = y * drop_x

### Conjunto de Features

In [376]:
x

,quant_pets,faixa_salarial,idade,peso,altura_do_pai
0,1,9267,51,94,1.61
1,0,6639,40,75,1.61
2,1,4167,25,85,1.69
3,0,5222,48,83,1.55
4,2,7151,73,99,1.75
5,1,4857,55,76,1.78
6,2,6109,32,84,1.69
7,1,6261,47,73,1.66
8,1,4204,33,96,1.61
9,0,4805,37,79,1.66


### Variável Resposta

In [377]:
y

0     1
1     0
2     0
3     0
4     1
5     0
6     1
7     1
8     0
9     0
10    0
11    1
12    1
13    0
14    0
dtype: int64

# Algoritmo boruta Implementação

O algoritmo apresenta 10 passos.

## 1-) Criação de shadow features

A partir do dataset inicial, criamos uma cópia para cada feature e a renomeamos com a palavra shadow.

In [454]:
num_of_features = x.shape[1]
x_shadow = x.copy()
x_shadow.columns = ['shadow_' + col_name for col_name in x.columns]

5


## 2-) Embaralhamento das shadow features

Em cada shadow feature, realizamos um embaralhamento dos seus dados. Isso é feito para remover a correlação
entre a shadow feature e a variável resposta. 

In [402]:
np.random.seed(21)
x_shadow = x_shadow.apply( lambda x: np.random.permutation(x) )
x_shadow = x_shadow.reset_index()
x_shadow.drop('index', axis=1, inplace=True)

### Juntamos as features iniciais com as shadow features

In [411]:
x_merged = pd.concat( [x, x_shadow], axis=1 )
x_merged.columns

Index(['quant_pets', 'faixa_salarial', 'idade', 'peso', 'altura_do_pai',
       'shadow_quant_pets', 'shadow_faixa_salarial', 'shadow_idade',
       'shadow_peso', 'shadow_altura_do_pai'],
      dtype='object')

## 3-) Treinamento do modelo com o novo conjunto de dados (features originais + shadow features)

Será usado o algoritmo Random Forest para treinar o modelo, pois este algoritmo calcula <br>
a importância das features durante o seu treinamento, esse cálculo é baseado no método de Gini.


In [413]:
rf = RandomForestClassifier(random_state=42, max_depth=3)
rf.fit( x_merged, y )

RandomForestClassifier(max_depth=3, random_state=42)

In [406]:
rf.feature_importances_

array([0.10343029, 0.35391345, 0.06762231, 0.04514903, 0.060671  ,
       0.03689565, 0.12619728, 0.05067433, 0.09926615, 0.05618051])

In [488]:
importance_table = pd.DataFrame([rf.feature_importances_], columns=x_merged.columns)

### Vamos obter a maior importância dentre as shadow features 

In [513]:
imp_feature_list = rf.feature_importances_
imp_original_feature_list = imp_feature_list[:len(x.columns)]
imp_shadow_feature_list = imp_feature_list[len(x.columns):]
print('O maior valor de importância dado a uma shadow feature foi: ' + str(np.round( imp_shadow_feature_list.max(), 2) ))

O maior valor de importância dado a uma shadow feature foi: 0.13


## 4-) Avaliação da importância das features originais com as shadow features

Se a feature original apresentar importância maior que a maior importância das shadow features, ela teve um sucesso.

Caso contrário, ela teve um fracasso.

In [511]:
sucess_table = ( imp_original_feature_list > imp_shadow_feature_list.max() ).astype(int)
sucess_table

array([0, 1, 0, 0, 0])

## 5-) Reembaralhamento das shadow features e repetição dos processos 3 e 4

Para obter uma melhor precisão acerca da importância fornecida pelo algoritmo Random Forest,<br>
repetimos o processo por 10 vezes, sempre marcando os pontos.

## Algoritmo Completo

# Decisão de manter ou excluir as features

Após o fim do treinamento,

# Bibliografia

 An introduction to feature selection - https://machinelearningmastery.com/an-introduction-to-feature-selection/